In [21]:
Authors: Raja, Divjang, Greg, and Brennan.

SyntaxError: invalid syntax (<ipython-input-21-c09e00f0c728>, line 1)

In this experiment, our hypotheseis is whether emojis can be mined in retweets and replies for sentiment on a particular issue.

In [22]:
import os
import re
from operator import add
from nltk import tokenize
import string
from vaderSentiment.vaderSentiment import sentiment as vaderSentiment

# Get current working directory
script_dir = os.getcwd()

# The relative path to the data files one for each timezone.
est_rel_path = "../../../tweet_data/tweets_est.psv"
mst_rel_path = "../../../tweet_data/tweets_mst.psv"
pst_rel_path = "../../../tweet_data/tweets_pst.psv"

# Absolute path to the datafiles
est_abs_file_path = os.path.join(script_dir, est_rel_path)
mst_abs_file_path = os.path.join(script_dir, mst_rel_path)
pst_abs_file_path = os.path.join(script_dir, pst_rel_path)

# Get the text of the tweet
def getTweet(line):
    line = line.strip('\n')
    fields = line.split('|')
    
    if 10 == len(fields):
        return fields[9]
    return ''
    
# Get the language that the tweet is written in
def getLang(line):
    # strip new line
    line = line.strip('\n')
    # split on the pipe
    fields =line.split('|')

    if 10 == len(fields) :
        
        val = fields[8]
        # strip leading and trailing spaces.
        val = val.lstrip()
        lang= val.rstrip()
        #skip the header
        if lang == 'lang':
            return ''
        # return a valid language
        return lang
    return ''
 
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

emoji_chars_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]", flags=re.UNICODE)
# The list of positive emoji icons
positive_emojis = [
    u"\U0001F601",
     u"\U0001F602",
     u"\U0001F603",
     u"\U0001F604",
     u"\U0001F605",
     u"\U0001F606",
     u"\U0001F609",
     u"\U0001F60A",
     u"\U0001F60B",
     u"\U0001F60C",
     u"\U0001F60D",  
     u"\U0001F618",
     u"\U0001F61A",
     u"\U0001F623",
     u"\U0001F624",
     u"\U0001F638",
     u"\U0001F639",
    u"\U0001F63A",
    u"\U0001F63B",
    u"\U0001F63C",
    u"\U0001F63D",
    u"\U0001F64C",
    u"\U0001F64B" 
]

# The list of negative emoji icons
negative_emojis = [
     u"\U0001F60F",
    u"\U0001F612",
    u"\U0001F613",
    u"\U0001F61E",
    u"\U0001F620",
    u"\U0001F622",
    u"\U0001F628",
    u"\U0001F629",
    u"\U0001F62A",
    u"\U0001F62B",
    u"\U0001F62D",
    u"\U0001F631",
    u"\U0001F64D",
    u"\U0001F64E",
     u"\U0001F63F",
    u"\U0001F63E",
    u"\U0001F640"     
]

 
# Find all the emoji characters in this line
def find_emoji_chars(line):
    
    line = line.strip('\n')
    emojis = emoji_chars_pattern.findall(line)
    return emojis

# This function determines the sentiment expressed by the emojis. It parses the
# tweet text and tries to determine the sentiment using th evaderSentiment package
# If language of this tweet is not "en" then it returns "notenglish".
# If there are no positive or negative emoticons then return "noemoticon"
# When sentiment expressed is not clear then it returns "MixedSentiment"
# If tweet is not RT or reply then returns "No-RT-or-reply"
# Otherwise return "SentimentExpressed".

def find_emoji_sentiment(line):
    
    # If language is not english then return. Sentiment analysis being done only
    # for english.
    if getLang(line) != 'en':
        return "notenglish"
    
    #Find all the emoji chars
    emojis = find_emoji_chars(line)
    
    # The counts of icons that represent positive, negative, and neutral emotions.
    pos = 0
    neg = 0
    neu = 0
    
    for em in emojis:
        if em in positive_emojis:
            pos += 1
        elif em in negative_emojis:
            neg += 1 
            
    # Return if there are no positive nor negative icons.
    if pos == 0 and neg == 0:
        return "noemoticon"
    
    # Determine the sentiment expressed by the emojis
    icon =""
    if neg > pos :
        icon = "neg"
    elif pos > neg :
        icon =  "pos"
    else:
        icon = "neu"
    
    # If the number of positive icons is same as the number of negativ eicons. Then the sentiment
    # expressed is not clear. Also when we have atleast one of each kind then it is hard to determine
    # the sentiment expressed. return "MixedSentiment"
    if pos != 0 and neg !=0:
        return "MixedSentiment"
    
    # Use vaderSentiment to compute the sentiment for replies and retweets.
    # strip newline
    line = line.strip("\n")
    # split on pipe
    fields = line.split("|")
    
    if len(fields) == 10:
        # text of the tweet
        tweet = getTweet(line)
        # strip leading and trailing space
        tweet = tweet.lstrip()
        tweet = tweet.rstrip()
        
        # strip the emojis if present in the text.
        for x in emojis:
            tweet = tweet.replace(x, " ")
        
        # strip other unicode chars
        tweet = re.sub('[^\x00-\x7f]', '', tweet)
         
        
        # If this is not a retweet or reply return "No-RT-or-reply"
        if not ('RT' in tweet or 'reply' in tweet):
            return "No-RT-or-reply"
        
        # Break the paragraph into lines.
        lines_list = tokenize.sent_tokenize(tweet)
        
        # maintain the result over the whole paragraph
        result = { 'pos':[] , 'neg':[], 'compound':[], 'neu':[] }
        
        # Compute sentiment for each line separately and sum it up.
        for item in lines_list:
            vs =vaderSentiment(item)
            result['pos'].append(vs['pos'])
            result['neg'].append(vs['neg'])
            result['compound'].append(vs['compound'])
            result['neu'].append(vs['neu'])

        for i in result.keys():
          sum(result[i])
        
        if result['pos'] > result['neg']:
            sent = "pos"
        elif  result['pos'] < result['neg']:
            sent ="neg"
        else:
            sent ="neu"
        
        return "SentimentExpressed"
        
         

# Load the data for EST      
estrdd = sc.textFile(est_abs_file_path)

print 'Eastern Timezone'
print
esttweetswithemoji = estrdd.filter(lambda line: emoji_pattern.findall(line) != [])

# The emoji sentiment
estemojisentiment  = esttweetswithemoji.map(lambda line: (find_emoji_sentiment(line), 1)).reduceByKey(add)
print '  Emoji Sentiment Analysis of Tweets'
print estemojisentiment.collect()

print
print 'Mountain Timezone'
print
# Load the data for MST
mstrdd     = sc.textFile(mst_abs_file_path)

msttweetswithemoji = mstrdd.filter(lambda line: emoji_pattern.findall(line) != [])
mstemojisentiment  = msttweetswithemoji.map(lambda line: (find_emoji_sentiment(line), 1)).reduceByKey(add)

print '  Emoji Sentiment Analysis of Tweets'
print mstemojisentiment.collect()


print
print 'Pacific Timezone'
print
# Load the data for PST
pstrdd     = sc.textFile(pst_abs_file_path)

psttweetswithemoji = pstrdd.filter(lambda line: emoji_pattern.findall(line) != [])

pstemojisentiment  = psttweetswithemoji.map(lambda line: (find_emoji_sentiment(line), 1)).reduceByKey(add)
print '  Emoji Sentiment Analysis of Tweets'
print pstemojisentiment.collect()



Eastern Timezone

  Emoji Sentiment Analysis of Tweets
[('No-RT-or-reply', 3210), ('noemoticon', 5079), ('MixedSentiment', 544), ('SentimentExpressed', 2825), ('notenglish', 3407)]

Mountain Timezone

  Emoji Sentiment Analysis of Tweets
[('notenglish', 5676), ('No-RT-or-reply', 4225), ('noemoticon', 4877), ('SentimentExpressed', 2508), ('MixedSentiment', 550)]

Pacific Timezone

  Emoji Sentiment Analysis of Tweets
[('SentimentExpressed', 1584), ('No-RT-or-reply', 2143), ('noemoticon', 3368), ('MixedSentiment', 307), ('notenglish', 5765)]
